# Performance Blow-out

## Description
When agents maximize their time when working on these activities by stretching their work, acting as if they are doing the work, or by falsely recording end times.

In [111]:
import pandas as pd
import numpy as np
import altair as alt

In [112]:
log_name = 'propr'

fn_logs = {
    'bpic15': '../data/preproc/bpic2015_disco.csv',
    'bpic17': '../data/preproc/bpic2017.csv',
    'propr': '../data/preproc/proprietary.csv'
}

In [113]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost
0,Approval_Branch,005141EEB1240B31C12577DF004F6A77,18/11/2010 15:27,18/11/2010 15:58,004-9-1,Thursday,0.0,2.0,0.0,"0,87 € per minute"
1,Precheck,005141EEB1240B31C12577DF004F6A77,19/11/2010 12:45,19/11/2010 12:46,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute"
2,Precheck,005141EEB1240B31C12577DF004F6A77,24/11/2010 8:18,24/11/2010 8:26,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute"
3,Check_of_Processing_Applications,005141EEB1240B31C12577DF004F6A77,24/11/2010 10:35,24/11/2010 10:35,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute"
4,Processing_of_Applications,005141EEB1240B31C12577DF004F6A77,2/12/2010 10:46,2/12/2010 10:46,010-23-11,Thursday,0.0,2.0,0.0,"1,02 € per minute"
...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 11:38,13/12/2010 11:40,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute"
18441,Check_of_Processing_Applications,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 12:06,13/12/2010 12:14,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute"
18442,Processing_of_Applications,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:48,22/12/2010 11:48,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute"
18443,Archieving,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:49,22/12/2010 13:19,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute"


In [114]:
# add activity duration
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], dayfirst=True)
log['start timestamp'] = pd.to_datetime(log['start timestamp'], dayfirst=True)
log['activity_duration_minute'] = (
    log['complete timestamp'] -
    log['start timestamp']
).dt.total_seconds() / 60
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,activity_duration_minute
0,Approval_Branch,005141EEB1240B31C12577DF004F6A77,2010-11-18 15:27:00,2010-11-18 15:58:00,004-9-1,Thursday,0.0,2.0,0.0,"0,87 € per minute",31.0
1,Precheck,005141EEB1240B31C12577DF004F6A77,2010-11-19 12:45:00,2010-11-19 12:46:00,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",1.0
2,Precheck,005141EEB1240B31C12577DF004F6A77,2010-11-24 08:18:00,2010-11-24 08:26:00,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",8.0
3,Check_of_Processing_Applications,005141EEB1240B31C12577DF004F6A77,2010-11-24 10:35:00,2010-11-24 10:35:00,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",0.0
4,Processing_of_Applications,005141EEB1240B31C12577DF004F6A77,2010-12-02 10:46:00,2010-12-02 10:46:00,010-23-11,Thursday,0.0,2.0,0.0,"1,02 € per minute",0.0
...,...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,FFFF329EF772D73CC12577EA00534A1C,2010-12-13 11:38:00,2010-12-13 11:40:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",2.0
18441,Check_of_Processing_Applications,FFFF329EF772D73CC12577EA00534A1C,2010-12-13 12:06:00,2010-12-13 12:14:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",8.0
18442,Processing_of_Applications,FFFF329EF772D73CC12577EA00534A1C,2010-12-22 11:48:00,2010-12-22 11:48:00,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",0.0
18443,Archieving,FFFF329EF772D73CC12577EA00534A1C,2010-12-22 11:49:00,2010-12-22 13:19:00,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",90.0


In [115]:
# Does "Cost" differ by activity or resource?
# NOTE: it seems that all resources are associated with a stable cost, except
# resource 004-9-1, which has two costs
# df = log[['resource', 'Cost']].groupby('resource').agg(cost_options=pd.NamedAgg('Cost', aggfunc='nunique'))
# df[df['cost_options'] > 1]


## Performance-PB-1

### Profile work by activity types and/or case types

In [116]:
# proprietary: 
# case types as types of loan
log['case_type'] = log['Type of loan; 0 = normal; 1 = special'].apply(
    lambda x: 'special loan' if x == 1 else 'normal loan'
)

case_types = list(log['case_type'].unique())
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,activity_duration_minute,case_type
0,Approval_Branch,005141EEB1240B31C12577DF004F6A77,2010-11-18 15:27:00,2010-11-18 15:58:00,004-9-1,Thursday,0.0,2.0,0.0,"0,87 € per minute",31.0,normal loan
1,Precheck,005141EEB1240B31C12577DF004F6A77,2010-11-19 12:45:00,2010-11-19 12:46:00,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",1.0,normal loan
2,Precheck,005141EEB1240B31C12577DF004F6A77,2010-11-24 08:18:00,2010-11-24 08:26:00,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",8.0,normal loan
3,Check_of_Processing_Applications,005141EEB1240B31C12577DF004F6A77,2010-11-24 10:35:00,2010-11-24 10:35:00,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",0.0,normal loan
4,Processing_of_Applications,005141EEB1240B31C12577DF004F6A77,2010-12-02 10:46:00,2010-12-02 10:46:00,010-23-11,Thursday,0.0,2.0,0.0,"1,02 € per minute",0.0,normal loan
...,...,...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,FFFF329EF772D73CC12577EA00534A1C,2010-12-13 11:38:00,2010-12-13 11:40:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",2.0,normal loan
18441,Check_of_Processing_Applications,FFFF329EF772D73CC12577EA00534A1C,2010-12-13 12:06:00,2010-12-13 12:14:00,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",8.0,normal loan
18442,Processing_of_Applications,FFFF329EF772D73CC12577EA00534A1C,2010-12-22 11:48:00,2010-12-22 11:48:00,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",0.0,normal loan
18443,Archieving,FFFF329EF772D73CC12577EA00534A1C,2010-12-22 11:49:00,2010-12-22 13:19:00,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",90.0,normal loan


### Determine groups of comparable resources

In [117]:
# proprietary: select resources who
# participated in the selected case type; and
# were involved in at least 100 cases
res_workload = list()
for res, events in log.groupby('resource'):
    total_num_cases = events['case_id'].nunique()
    for ct in case_types:
        res_workload.append({
            'resource': res, 
            'case_type': ct,
            'frequency': (
                events.loc[events['case_type'] == ct, 'case_id'].nunique()
            ),
            'total_num_cases': total_num_cases
        })

res_workload = pd.DataFrame(res_workload)

sel_resources = set(res_workload.loc[
    (res_workload['case_type'] == 'special loan') & 
    (res_workload['frequency'] > 0) &
    (res_workload['total_num_cases'] >= 100),
    'resource'
].unique())

### Profile group member performance by activity duration

In [118]:
# use average activity duration for the same activity
res_performance = log.groupby(
    ['resource', 'activity', 'case_type']
).agg(
    mean_activity_duration_minute=pd.NamedAgg(
        'activity_duration_minute', 'mean'
    )
).reset_index()
res_performance = res_performance[
    res_performance['resource'].isin(sel_resources)
]
res_performance

,resource,activity,case_type,mean_activity_duration_minute
19,000-2-01,Application_Processing_Branches,normal loan,9.873754
20,000-2-01,Application_Processing_Branches,special loan,13.666667
21,000-2-01,Approval,normal loan,6.105263
22,000-2-01,Archieving,normal loan,3.650000
23,000-2-01,Check_of_Processing_Applications,normal loan,5.672078
...,...,...,...,...
835,070-21-01,Approval_Branch,special loan,56.000000
836,070-21-01,Precheck,normal loan,3.756579
837,070-21-01,Precheck,special loan,6.086957
838,070-21-01,Refusal_Branches,normal loan,7.466667


### Identify resources who spent significantly longer time on the same type of work compared to others

In [119]:
data = res_performance[
    (res_performance['resource'].isin(sel_resources)) &
    (res_performance['case_type'] == 'special loan')
]
alt.Chart(data).mark_rect().encode(
    y=alt.Y('resource:N').title('Resource'),
    x=alt.X('activity:N').title('Activity'),
    color=alt.Color('mean_activity_duration_minute').title(
        'Average activity duration (minutes)'
    ),
    tooltip=['resource', 'activity', 'case_type', 'mean_activity_duration_minute']
)

alt.Chart(...)

In [124]:
data = res_performance[
    (res_performance['resource'].isin(sel_resources)) &
    (res_performance['case_type'] == 'special loan')
]

charts = []
for activity in data['activity'].unique():
    subset = data[data['activity'] == activity]
    charts.append(
        alt.layer(
            alt.Chart(subset).mark_boxplot().encode(
                y='activity',
                x='mean_activity_duration_minute',
                opacity=alt.value(0.2)
            ),
            alt.Chart(subset).mark_circle().encode(
                y='activity',
                x='mean_activity_duration_minute',
                color='resource',
                tooltip=['resource', 'activity', 'mean_activity_duration_minute']
            )
        )
    )
alt.vconcat(*charts)

alt.VConcatChart(...)